In [ ]:
#Deep Learning Code
#Importing related packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
from itertools import chain
import os
import random
import zipfile
import json

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import timm

import torchvision
import matplotlib.pyplot as plt

from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, utils
from tqdm.notebook import tqdm
print(f"Torch: {torch.__version__}")

import gc
import random
from PIL import Image,ImageFilter

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

gc.collect()

#Empty the GPU
torch.cuda.empty_cache()

device = 'cuda'

#Add pretzel noise
class AddSaltPepperNoise(object):

    def __init__(self, density=0,p=0.5):
        self.density = density
        self.p = p

    def __call__(self, img):
        if random.uniform(0, 1) < self.p:  # probabilistic judgement
            img = np.array(img)  # Image to numpy
            h, w, c = img.shape
            Nd = self.density
            Sd = 1 - Nd
            mask = np.random.choice((0, 1, 2), size=(h, w, 1), p=[Nd / 2.0, Nd / 2.0, Sd])  # Generate a mask for a channel
            mask = np.repeat(mask, c, axis=2)  # Duplicate in the dimension of the channel to generate a coloured mask
            img[mask == 0] = 0  
            img[mask == 1] = 255  
            img = Image.fromarray(img.astype('uint8')).convert('RGB')  # numpy to image
            return img
        else:
            return img

#Adding Gaussian Noise
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, variance=1.0, amplitude=1.0):

        self.mean = mean
        self.variance = variance
        self.amplitude = amplitude

    def __call__(self, img):

        img = np.array(img)
        h, w, c = img.shape
        N = self.amplitude * np.random.normal(loc=self.mean, scale=self.variance, size=(h, w, 1))
        N = np.repeat(N, c, axis=2)
        img = N + img
        img[img > 255] = 255                       
        img = Image.fromarray(img.astype('uint8')).convert('RGB')
        return img

#Add Blur
class Addblur(object):

    def __init__(self, p=0.5,blur="normal"):
        # self.density = density
        self.p = p
        self.blur= blur

    def __call__(self, img):
        if random.uniform(0, 1) < self.p:  # 概率的判断
       		 #standard ambiguity
            if self.blur== "normal":
                img = img.filter(ImageFilter.BLUR)
                return img
            #Gaussian blur
            if self.blur== "Gaussian":
                img = img.filter(ImageFilter.GaussianBlur)
                return img
            #mean value ambiguity
            if self.blur== "mean":
                img = img.filter(ImageFilter.BoxBlur)
                return img

        else:
            return img

#Append data from each epoch to an Excel file
def append_epoch_data_to_excel(epoch, train_loss, train_acc, val_loss, val_acc, excel_path):
    epoch_data = pd.DataFrame({
        'Epoch': [epoch],
        'Train Loss': [train_loss],
        'Train Acc': [train_acc],
        'Val Loss': [val_loss],
        'Val Acc': [val_acc]
    })
    try:
        with pd.ExcelWriter(excel_path, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            epoch_data.to_excel(writer, index=False, header=False, startrow=writer.sheets['Sheet1'].max_row)
    except FileNotFoundError:
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            epoch_data.to_excel(writer, index=False)
            
#Data Enhancement:
train_transforms = transforms.Compose([
    transforms.RandomOrder([  # random order
        # random rotation
        transforms.RandomApply([transforms.RandomRotation([-30, 30])], p=0.3),
        # Horizontal Flip
        transforms.RandomHorizontalFlip(p=0.3),
        # Random cropping and scaling
        transforms.RandomApply([transforms.RandomResizedCrop((150, 200), scale=(0.4, 0.8), ratio=(1./5., 5./1.))], p=0.5),
        # Randomly changing image properties: brightness, contrast, saturation and hue
        transforms.RandomApply([transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)], p=0.5),
        # random blur
        transforms.RandomApply([transforms.Compose([Addblur(p=1, blur="Gaussian")])], p=0.5),
        # Randomly added pretzel noise
        transforms.RandomApply([transforms.Compose([AddSaltPepperNoise(0.05, 1)])], p=0.5),
        # Add random Gaussian noise
        transforms.RandomApply([transforms.Compose([AddGaussianNoise(mean=random.uniform(0.5, 1.5), variance=0.5, amplitude=random.uniform(0, 45))])], p=0.5),
    ]),
    transforms.Resize((224, 224)),  
    # Converting images to PyTorch tensors
    transforms.ToTensor(),
    # Normalisation based on ImageNet mean and standard deviation
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


val_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

# Training settings
batch_size = 32
epochs = 50
lr = 2e-5
gamma = 0.7
seed = 42

train_dir = 'train'
val_dir = 'validation'
test_dir = 'test'

#Load Dataset
train_img_data = torchvision.datasets.ImageFolder(train_dir,
                                            transform=train_transforms,
                                            )

val_img_data = torchvision.datasets.ImageFolder(val_dir,
                                            transform=val_transforms,
                                            )
test_img_data = torchvision.datasets.ImageFolder(test_dir,
                                            transform=test_transforms,
                                            )

train_data_loader = torch.utils.data.DataLoader(train_img_data, batch_size=batch_size, shuffle=True, num_workers=8)
val_data_loader = torch.utils.data.DataLoader(val_img_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_data_loader = torch.utils.data.DataLoader(test_img_data, batch_size=batch_size, shuffle=True, num_workers=8)

print(len(train_data_loader.dataset))
print(len(val_data_loader.dataset))
print(len(test_data_loader.dataset))

# Get a list of all model names
model_list = timm.list_models()

#Getting the model
model = timm.create_model('swin_small_patch4_window7_224', pretrained=False, num_classes=50).to(device)
          
# loss function
criterion = nn.CrossEntropyLoss()

# optimizer 
optimizer = optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=25, eta_min=1e-6)

#Training Models
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    #for data, label in tqdm(train_loader):
    for data, label in tqdm(train_data_loader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_data_loader)
        epoch_loss += loss / len(train_data_loader)
        
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in val_data_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(val_data_loader)
            epoch_val_loss += val_loss / len(val_data_loader)
            
    
    excel_path = "swin_small_patch4_window7_224.xlsx"  
    append_epoch_data_to_excel(
    epoch, 
    epoch_loss.item(),  
    epoch_accuracy.item(), 
    epoch_val_loss.item(), 
    epoch_val_accuracy.item(), 
    excel_path)
    print(
            f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )

    torch.save(model, './swin_small_patch4_window7_224.pt')  # Save the model

In [ ]:
#超参数优化
!pip install --upgrade typing_extensions
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
from itertools import chain
import random
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset as torch_Dataset  # Renamed to avoid conflict with data module
from torchvision import datasets, transforms, utils
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import timm
import json
import gc
import matplotlib.pyplot as plt
import optuna
import torchvision
from sklearn.metrics import f1_score, recall_score, accuracy_score  # Combined import for sklearn metrics

gc.collect()

#Empty the GPU
torch.cuda.empty_cache()

device = 'cuda'

#Add pretzel noise
class AddSaltPepperNoise(object):

    def __init__(self, density=0,p=0.5):
        self.density = density
        self.p = p

    def __call__(self, img):
        if random.uniform(0, 1) < self.p:  # probabilistic judgement
            img = np.array(img)  # Image to numpy
            h, w, c = img.shape
            Nd = self.density
            Sd = 1 - Nd
            mask = np.random.choice((0, 1, 2), size=(h, w, 1), p=[Nd / 2.0, Nd / 2.0, Sd])  # Generate a mask for a channel
            mask = np.repeat(mask, c, axis=2)  # Duplicate in the dimension of the channel to generate a coloured mask
            img[mask == 0] = 0  
            img[mask == 1] = 255  
            img = Image.fromarray(img.astype('uint8')).convert('RGB')  # numpy to image
            return img
        else:
            return img

#Adding Gaussian Noise
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, variance=1.0, amplitude=1.0):

        self.mean = mean
        self.variance = variance
        self.amplitude = amplitude

    def __call__(self, img):

        img = np.array(img)
        h, w, c = img.shape
        N = self.amplitude * np.random.normal(loc=self.mean, scale=self.variance, size=(h, w, 1))
        N = np.repeat(N, c, axis=2)
        img = N + img
        img[img > 255] = 255                       
        img = Image.fromarray(img.astype('uint8')).convert('RGB')
        return img

#Add Blur
class Addblur(object):

    def __init__(self, p=0.5,blur="normal"):
        # self.density = density
        self.p = p
        self.blur= blur

    def __call__(self, img):
        if random.uniform(0, 1) < self.p:  # 概率的判断
       		 #standard ambiguity
            if self.blur== "normal":
                img = img.filter(ImageFilter.BLUR)
                return img
            #Gaussian blur
            if self.blur== "Gaussian":
                img = img.filter(ImageFilter.GaussianBlur)
                return img
            #mean value ambiguity
            if self.blur== "mean":
                img = img.filter(ImageFilter.BoxBlur)
                return img

        else:
            return img

#Append data from each epoch to an Excel file
def append_epoch_data_to_excel(epoch, train_loss, train_acc, val_loss, val_acc, excel_path):
    epoch_data = pd.DataFrame({
        'Epoch': [epoch],
        'Train Loss': [train_loss],
        'Train Acc': [train_acc],
        'Val Loss': [val_loss],
        'Val Acc': [val_acc]
    })
    try:
        with pd.ExcelWriter(excel_path, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            epoch_data.to_excel(writer, index=False, header=False, startrow=writer.sheets['Sheet1'].max_row)
    except FileNotFoundError:
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            epoch_data.to_excel(writer, index=False)
            
# Data enhancement parameters adjusted by Optuna            
def optimize_hyperparameters(trial):
    lr = trial.suggest_uniform('lr', 0, 1e-4)
    bn = trial.suggest_int('batchsize', 8, 52)
    T_max=trial.suggest_int('T_max', 5, 25)
    weight_decay=trial.suggest_uniform('weight_decay', 1e-3, 1e-1)
    p_RandomRotation = trial.suggest_float('p_RandomRotation', 0.1, 1)
    p_RandomHorizontalFlip = trial.suggest_float('p_RandomHorizontalFlip', 0.1, 1)
    crop_size_h = trial.suggest_int('crop_size_h', 150, 200)  
    crop_size_w = trial.suggest_int('crop_size_w', 150, 200)  
    p_RandomResizedCrop = trial.suggest_float('p_RandomResizedCrop', 0.1, 1)
    brightness = trial.suggest_float('brightness', 0.1, 0.5)
    contrast = trial.suggest_float('contrast', 0.1, 0.5)
    saturation = trial.suggest_float('saturation', 0.1, 0.5)
    hue = trial.suggest_float('hue', 0.1, 0.5)
    p_ColorJitter = trial.suggest_float('p_ColorJitter', 0.1, 1)  
    p_Addblur = trial.suggest_float('p_Addblur', 0.1, 1)
    p_AddSaltPepperNoise = trial.suggest_float('p_AddSaltPepperNoise', 0.1, 1)
    mean_GaussianNoise = trial.suggest_float('mean_GaussianNoise', 0.1, 1)
    variance_GaussianNoise = trial.suggest_float('variance_GaussianNoise', 0.1, 0.9)
    amplitude_GaussianNoise = trial.suggest_float('amplitude_GaussianNoise', 0.1, 1)
    p_AddGaussianNoise = trial.suggest_float('p_AddGaussianNoise', 0.1, 1)  
    
    train_transforms = transforms.Compose([
        transforms.RandomOrder([
            transforms.RandomApply([transforms.RandomRotation([-30, 30])], p=p_RandomRotation),
            transforms.RandomHorizontalFlip(p=p_RandomHorizontalFlip),
            transforms.RandomApply([transforms.RandomResizedCrop((crop_size_h, crop_size_w), scale=(0.5, 1.0), ratio=(1./5., 5./1.))], p=p_RandomResizedCrop),
            transforms.RandomApply([transforms.ColorJitter(brightness=brightness, contrast=contrast, saturation=saturation, hue=hue)], p=p_ColorJitter),
            transforms.RandomApply([Addblur(p=p_Addblur, blur="Gaussian")], p=p_Addblur),
            transforms.RandomApply([AddSaltPepperNoise(density=0.05, p=1)], p=p_AddSaltPepperNoise),
            transforms.RandomApply([AddGaussianNoise(mean=mean_GaussianNoise, variance=variance_GaussianNoise, amplitude=amplitude_GaussianNoise)], p=p_AddGaussianNoise),
        ]),
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    val_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )

    test_transforms = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )

#Importing data sets
    train_dir = 'train'
    val_dir = 'validation'
    test_dir = 'test'

    train_img_data = torchvision.datasets.ImageFolder(train_dir,
                                                transform=train_transforms,
                                                )

    val_img_data = torchvision.datasets.ImageFolder(val_dir,
                                                transform=val_transforms,
                                                )
    test_img_data = torchvision.datasets.ImageFolder(test_dir,
                                                transform=test_transforms,
                                                )

    train_data_loader = torch.utils.data.DataLoader(train_img_data, batch_size=bn, shuffle=True, num_workers=8)
    val_data_loader = torch.utils.data.DataLoader(val_img_data, batch_size=bn, shuffle=True, num_workers=8)
    test_data_loader = torch.utils.data.DataLoader(test_img_data, batch_size=bn, shuffle=True, num_workers=8)

    print(len(train_data_loader.dataset))
    print(len(val_data_loader.dataset))
    print(len(test_data_loader.dataset))

    model = timm.create_model("swin_small_patch4_window7_224", pretrained=False, num_classes=50).to(device)
   
    # loss function
    criterion = nn.CrossEntropyLoss()
    
    # optimizer
    optimizer = optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max, eta_min=1e-6)

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        epoch_accuracy = 0

        #for data, label in tqdm(train_loader):
        for data, label in tqdm(train_data_loader):
            data = data.to(device)
            label = label.to(device)

            output = model(data)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            acc = (output.argmax(dim=1) == label).float().mean()
            epoch_accuracy += acc / len(train_data_loader)
            epoch_loss += loss / len(train_data_loader)

        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for data, label in val_data_loader:
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = (val_output.argmax(dim=1) == label).float().mean()
                epoch_val_accuracy += acc / len(val_data_loader)
                epoch_val_loss += val_loss / len(val_data_loader)

        
        excel_path = "training_data_swin_small_patch4_window7_224_5.xlsx"  
        append_epoch_data_to_excel(
    epoch, 
    epoch_loss.item(),  
    epoch_accuracy.item(), 
    epoch_val_loss.item(), 
    epoch_val_accuracy.item(), 
    excel_path)
        print(
            f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )

    model_save_path = f"swin_small_patch4_window7_224_trial_{trial.number}_epoch_{epoch+1}.pt"
    torch.save(model.state_dict(), model_save_path)

    # return the validation accuracy as the objective value
    return epoch_val_loss

#Preservation of experimental results
def save_trial_to_excel(study, trial):
    trial_data = trial.params
    trial_data['trial_number'] = trial.number
    trial_data['epoch_val_loss'] = trial.value
    df_trial = pd.DataFrame([trial_data])
    
    excel_path = "optuna_trials_swin_small_patch4_window7_224_5.xlsx"

    try:
        with pd.ExcelWriter(excel_path, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            df_trial.to_excel(writer, index=False, header=False, startrow=writer.sheets['Sheet1'].max_row)
    except FileNotFoundError:
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            df_trial.to_excel(writer, index=False)

#Defining parameters
epochs=50
torch.manual_seed(42)            

#Perform a hyperparametric search
study = optuna.create_study()
study.optimize(lambda trial: optimize_hyperparameters(trial), n_trials=50, callbacks=[save_trial_to_excel])

# Save the results of the optimisation test to another Excel file.
best_trial = study.best_trial
best_trial_data = best_trial.params
best_trial_data['trial_number'] = best_trial.number
best_trial_data['epoch_val_loss'] = best_trial.value
df_best_trial = pd.DataFrame([best_trial_data])

excel_path_best_trial = "optuna_best_trial_swin_small_patch4_window7_224.xlsx"
df_best_trial.to_excel(excel_path_best_trial, index=False)

print(f"All trials are saved to optuna_trials.xlsx after each trial.")
print(f"Best trial is saved to optuna_best_trial.xlsx.")

print("Best trial:")
trial = study.best_trial
print("epoch_val_loss: ", trial.value)
print("Hyperparameters: ", trial.params)

In [ ]:
#Model Evaluation Code
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve

#Loading Models
model_path = 'swin_small_patch4_window7_224.pt'
model = torch.load(model_path, map_location=device)

# Set to evaluation mode
model.eval() 
prob_all = []
lable_all = []
prob_score=[]
i=0
for data, label in tqdm(test_data_loader):
        data = data.to(device)
        label = label.to(device)
        l2 = label.cpu().detach().numpy()
        lable_all.extend(l2)
        prob = model(data) 
        prob = prob.cpu().detach().numpy() 
        prob_score.extend(prob) 
        prob_all.extend(np.argmax(prob,axis=1))
        i=i+1
print(i)

root_dir = "test"

#Importing dataset labels
try:
    target_names = sorted([name for name in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, name))])
    print("Sequentially imported dataset labels (folders):", target_names)
except Exception as e:
    print("Error importing dataset labels:", str(e))

print("Accuracy:{:.4f}".format(accuracy_score(lable_all, prob_all) ))
print("Recall:{:.4f}".format(recall_score(lable_all, prob_all,average='macro') ))
print("Precision:{:.4f}".format(precision_score(lable_all, prob_all,average='macro') ))
print("f1_score:{:.4f}".format(f1_score(lable_all, prob_all,average='macro') ))

print(classification_report(lable_all,prob_all,target_names=target_names,digits=4))
print(confusion_matrix(prob_all,lable_all))

In [ ]:
#confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt

#Obtain the confusion matrix
conf_mat=confusion_matrix(prob_all,lable_all)
root_dir='test'

#Importing dataset categories
try:
    class_label = sorted([name for name in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, name))])
    print("Sequentially imported dataset labels (folders):", class_label)
except Exception as e:
    print("Error importing dataset labels:", str(e))
    
df_cm = pd.DataFrame(conf_mat, index=class_label, columns=class_label)
df_cm

#Mapping the confusion matrix
plt.figure(figsize=(18, 12))

heatmap=sns.heatmap(df_cm, annot=True, fmt="d",cmap="YlGnBu")
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(),rotation=0, ha='right')
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(),rotation=45, ha='right')
plt.title('Confusion Matrix for Test Set Using Swin transformer Model')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig("i_swin_small_patch4_window7_224", bbox_inches='tight', dpi=500)

In [ ]:
# t-SNE
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, sampler, random_split

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os
import random
import shutil
import time

import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.decomposition import PCA

root_dir = "test"

# Importing dataset labels
try:
    target_names = sorted([name for name in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, name))])
    print("Sequentially imported dataset labels (folders):", target_names)
except Exception as e:
    print("Error importing dataset labels:", str(e))

# Loading Models
model_path = 'swin_small_patch4_window7_224.pt'
model = torch.load(model_path, map_location=device)

# Get the predicted values of the training set
def get_representations(model, iterator):

    model.eval()

    outputs = []
    intermediates = []
    labels = []

    with torch.no_grad():
        for data, label in tqdm(iterator):
            data = data.to(device)
            #label = label.to(device)
            l2 = label.cpu().detach().numpy()
            lable_all.extend(l2)
            prob = model(data) 
            outputs.append(prob.cpu())
            labels.append(label)
        

    outputs = torch.cat(outputs, dim = 0)
    labels = torch.cat(labels, dim = 0)

    return outputs, labels

outputs, labels = get_representations(model, test_data_loader)

# t-SNE
def get_tsne(data, n_components = 2, n_images = None):
    
    if n_images is not None:
        data = data[:n_images]
        
    tsne = manifold.TSNE(n_components = n_components, early_exaggeration=3, random_state = 8, perplexity=70,learning_rate=300)
    tsne_data = tsne.fit_transform(data)
    return tsne_data

# Mapping t-SNE
def plot_representations(data, labels, classes, n_images=None):
    if n_images is not None:
        data = data[:n_images]
        labels = labels[:n_images]

    fig = plt.figure(figsize=(16, 10))  
    ax = fig.add_subplot(111)

    # Create enough colours to match the number of categories
    cmap = plt.get_cmap('nipy_spectral')
    colors = [cmap(i) for i in np.linspace(0, 1, len(classes))]

    # Plotting a scatterplot for each category
    for i, class_name in enumerate(classes):
        idx = labels == i  
        ax.scatter(data[idx, 0], data[idx, 1], label=class_name, color=colors[i])

    plt.title('VGG11 Model t-SNE Scatter Plot on Test Set')
    plt.xlabel('t-SNE Dim 1')
    plt.ylabel('t-SNE Dim 2')
    # Place the legend on the right side outside the graphic
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize='x-small', markerscale=2)
    # Resize the main image to make room for the legend
    plt.tight_layout(rect=[0, 0, 0.75, 1])  
    plt.savefig("i_t-SNE_VGG11_newdata_2", bbox_inches='tight', dpi=500)
    plt.show()
    
N_IMAGES = 6_000    
output_tsne_data = get_tsne(outputs)
plot_representations(output_tsne_data, labels,target_names)

In [ ]:
# DER+LwF code
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from torch.optim.lr_scheduler import StepLR
from torch.optim import SGD

import torchvision
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder

from PIL import Image, ImageFilter

from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import timm

from datetime import datetime

# Importing the necessary components of the avalanche library
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.training import DER
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics,bwt_metrics,forgetting_metrics,forward_transfer_metrics
from avalanche.logging import InteractiveLogger, TensorboardLogger,CSVLogger,TextLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins import LwFPlugin

gc.collect()

torch.cuda.empty_cache()  

# Setting parameters
lr = 0.0000248597317274012
batch_size = 37

#Add pretzel noise
class AddSaltPepperNoise(object):

    def __init__(self, density=0,p=0.5):
        self.density = density
        self.p = p

    def __call__(self, img):
        if random.uniform(0, 1) < self.p:  # probabilistic judgement
            img = np.array(img)  # Image to numpy
            h, w, c = img.shape
            Nd = self.density
            Sd = 1 - Nd
            mask = np.random.choice((0, 1, 2), size=(h, w, 1), p=[Nd / 2.0, Nd / 2.0, Sd])  # Generate a mask for a channel
            mask = np.repeat(mask, c, axis=2)  # Duplicate in the dimension of the channel to generate a coloured mask
            img[mask == 0] = 0  
            img[mask == 1] = 255  
            img = Image.fromarray(img.astype('uint8')).convert('RGB')  # numpy to image
            return img
        else:
            return img

#Adding Gaussian Noise
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, variance=1.0, amplitude=1.0):

        self.mean = mean
        self.variance = variance
        self.amplitude = amplitude

    def __call__(self, img):

        img = np.array(img)
        h, w, c = img.shape
        N = self.amplitude * np.random.normal(loc=self.mean, scale=self.variance, size=(h, w, 1))
        N = np.repeat(N, c, axis=2)
        img = N + img
        img[img > 255] = 255                       
        img = Image.fromarray(img.astype('uint8')).convert('RGB')
        return img

#Add Blur
class Addblur(object):

    def __init__(self, p=0.5,blur="normal"):
        # self.density = density
        self.p = p
        self.blur= blur

    def __call__(self, img):
        if random.uniform(0, 1) < self.p:  # 概率的判断
       		 #standard ambiguity
            if self.blur== "normal":
                img = img.filter(ImageFilter.BLUR)
                return img
            #Gaussian blur
            if self.blur== "Gaussian":
                img = img.filter(ImageFilter.GaussianBlur)
                return img
            #mean value ambiguity
            if self.blur== "mean":
                img = img.filter(ImageFilter.BoxBlur)
                return img

        else:
            return img

# Define a function for saving results in real time
def save_results_to_file(results, base_filename='results'):
    # Get the current date and time, formatted as a string
    current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f'{base_filename}_{current_time}.txt'  # 构造文件名
    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)
        
# Image preprocessing for training and validation
train_transforms = transforms.Compose([
   transforms.RandomOrder([  
        transforms.RandomApply([transforms.RandomRotation([-30, 30])], p=0.763308964823317),
        transforms.RandomHorizontalFlip(p=0.276583598372167),
        transforms.RandomApply([transforms.RandomResizedCrop((163, 162), scale=(0.4, 0.8), ratio=(1./5., 5./1.))], p=0.298899012089413),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.385039910829659, contrast=0.469248558722806, saturation=0.222816942158365, hue=0.332066381812662)], p=0.577596666960827),
        transforms.RandomApply([transforms.Compose([Addblur(p=1, blur="Gaussian")])], p=0.321736665705054),
        transforms.RandomApply([transforms.Compose([AddSaltPepperNoise(0.05, 1)])], p=0.155074274460154),
        transforms.RandomApply([transforms.Compose([AddGaussianNoise(mean=0.432936311664746, variance=0.142237563518334, amplitude=0.34620281978491)])], p=0.489988627632719),
    ]),
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dir = 'train'
test_dir = 'test'

train_dataset = torchvision.datasets.ImageFolder(root=train_dir, transform=train_transforms)
test_dataset = torchvision.datasets.ImageFolder(root=test_dir, transform=test_transforms)

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

print(f"train dataset size: {len(train_data_loader.dataset)}")
print(f"Test dataset size: {len(test_data_loader.dataset)}")

# Create class incremental learning scenarios
scenario = nc_benchmark(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    n_experiences=5,  
    task_labels=False,
    shuffle=True,
    seed=1234,
)

#Setting the path to save the file
tensorboard_log_dir = "path_to_your_tensorboard_log_dir_swin" 
csv_log_folder = "path_to_your_csv_log_folder_swin"  

# Setting the log file path
text_log_file_path = "path_to_your_text_log_file_swin.txt"

# Create an instance of TextLogger, specifying the output file
text_logger = TextLogger(file=open(text_log_file_path, "w"))

# Define evaluation plug-ins
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True, trained_experience=True),
    loss_metrics(epoch=True, experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    loggers=[InteractiveLogger(), TensorboardLogger(tensorboard_log_dir),CSVLogger(csv_log_folder),text_logger]
)

lwf_plugin = LwFPlugin(alpha=[0, 1 / 2, 2 * (2 / 3), 3 * (3 / 4), 4 * (4 / 5)], temperature=2)

# Loading Models
model = timm.create_model('swin_small_patch4_window7_224', pretrained=False, num_classes=50)
model.to('cuda')

# Configuration Optimiser
optimizer = optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)

# Initialising a policy instance
strategy = DER(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    mem_size = 200, 
    batch_size_mem = 10, 
    alpha = 0.1, 
    beta = 0.5, 
    train_mb_size=64,
    train_epochs=50,
    eval_mb_size=100,
    device='cuda',
    plugins=[lwf_plugin],
    # Pass in the eval_plugin instance
    evaluator=eval_plugin,  
)
    
print("Starting experiment...")
results = []
for experience in scenario.train_stream:
    print("Start of experience ", experience.current_experience)
    strategy.train(experience, num_workers=4)
    print("Training completed")

    print("Computing accuracy on the whole test set")
    # Save results after each assessment
    current_result = strategy.eval(scenario.test_stream, num_workers=8)
    results.append(current_result)
    # Calling a function to save results in real time
    save_results_to_file(results, 'DER_lwf_realtime_results_swin')  

print(f"Test metrics:\n{results}")

torch.save(model, './swin_DER_lwf.pt')

# Save results to file
results_str = json.dumps(results, indent=4)

# Eventually save again to make sure the final result is also saved
save_results_to_file(results, 'DER_lwf_final_results_swin')